In [ ]:
!pip install estnltk==1.7.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.2/222.2 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
tqdm.pandas()

from estnltk import Text
from estnltk.taggers import SpellCheckRetagger

In [ ]:
url = 'https://github.com/markmets/immigration-prediction-EST/raw/main/Annotated_Dataset.csv'
data = pd.read_csv(url)
columns = ", ".join(list(data.columns))

print(f"Sentences: {len(data)}")
print(f"Columns: {columns}")

Sentences: 7435
Columns: sentence, stanceConsolidated, stanceNotConsolidated, coderConsolidated, coderNotConsolidated, publishDate, title, publisherGroup, publisher, category, keywordGroups, keywords, author


* **sentence** - not always one sentence
* **stanceConsolidated** - was there a stance or not (scale 1-5 + ambiguous)
* **publishDate, title, publisherGroup, category, keywordGroups, keywords, author** - article meta features

From Mets et al.
https://arxiv.org/ftp/arxiv/papers/2305/2305.13047.pdf

_"The sentences were annotated on a 1-5 point scale from pro- to anti-immigration, with the option to mark the sentence as ambiguous instead. Ratings were later reduced to four classes of Against (1-2), Neutral (3), Supportive (4-5), and Ambiguous."_

In [ ]:
data.head(50)

,sentence,stanceConsolidated,stanceNotConsolidated,coderConsolidated,coderNotConsolidated,publishDate,title,publisherGroup,publisher,category,keywordGroups,keywords,author
0,"Ja selliseid inimesi on Eesti võimukoridorides ohtralt!Veel hullem on kogu see maailma päästmine, kus meile on Brüsseli käsul oma huvidest tähtsamad Süüria sõjapõgenikud, üle Vahemere tulevad neegritest majandusimmigrandid, Lääne-Euroopas välja mõeldud geiõigused ja sooneutraalsus, mujalt siia tulevad töölised ja nii edasi – ainult mitte omad inimesed.",1,1,A,J,29.05.2018 08:41,"Eesti riigivõim ei ehita oma riiki, vaid tegeleb Suure Hundi käsul maailma parandamisega",UuedUudised,UuedUudised,Arvamus,"['ränne', 'pagulased', 'rass']","['migra', 'põgenik', '([\\W ]|^)neeg']",NaN
1,Massiimmigratsioon oleks Euroopale hukatuslik ja see ei lahendaks maailmas mitte midagi.2018.,2,2,A,J,13.10.2020 08:31,Aafriklaste massiränne Euroopasse tähendab Musta Mandri kolimist külmemasse kliimasse ja muud ei midagi,UuedUudised,UuedUudised,Maailm,['ränne'],['migra'],NaN
2,"Lõppjäreldus on, et Eesti rahvaarv kasvab võõraste sisserändest, mis on rahvusriigile ohtlik.“Iga-aastase paarituhandelise sisserände ülekaalu puhul hakkaks eestlaste suhtarv rahvastikus aegamööda vähenema.",2,3,A,J,21.01.2019 10:22,"Eesti rahvaarv suureneb, kuid rahvusriigina pole põhjust hõisata",UuedUudised,UuedUudised,Eesti,['ränne'],['rände'],NaN
3,"Kui meeleavaldajad hakkavad liikuma, organiseerijate palvel kõhedusttekitavalt vaikides, ühineb pensionär nendega.ee juhtus demonstratsioonil, mille korraldas PEGIDA ehk Patriotische Europäer gegen die Islamisierung des Abendlandes (patriootlikud eurooplased õhtumaa islamiseerimise vastu).",3,MH,A,J,06.01.2015 01:30,Dresdeni rahvahulgad peegeldavad Euroopa hirmu migrantide ees,EkspressGrupp,Eesti Päevaleht,NaN,['rahvus'],['islam'],NaN
4,"See sõda tähendab, et peame ka Eestis oma julgeolekus arvesse võtma islami ekstremismi ohu hoopis uut olukorda silmas pidades."".",3,MH,A,J,14.11.2015 13:21,"Reinsalu: lähiajal karmistatakse terrorivastaseid seadusi, äärmuslastel ei tohi olla mingit kohta oma mürgi külvamisel",EkspressGrupp,Delfi,NaN,['rahvus'],['islam'],NaN
5,"”“Läbimõeldud, kogukonna juhitud ja täielikult ressurssidega kaetud tegevuskava, mis tegeleb mustanahaliste konkreetsete väljakutsetega, peab olema esmatähtis,” kirjutas Cullors ja lisas, et “me tahame oma häälte eest midagi.”BLM-i asutaja väitis koguni, et Biden ja Harris võitsid 2020. aasta presidendivalimised tänu mustanahalistele – seda hoolimata sellest, et valimistulemused pole veel kinnitatud, samuti hoolimata asjaolust, et president Donald Trump pälvis tegelikult suure osa mustanahaliste valijate toetuse, saades mustanahaliste häältest varasema 13 protsendi asemel 18 protsenti.„Ilma mustanahaliste suure toetuseta oleks valimistel hoopis teised tulemused.",3,MH,A,J,12.11.2020 14:15,"“Black Lives Matter” võttis Bideni ja Harrise pantvangi, nõudes võimu jagamist toetuse eest valimistel",UuedUudised,UuedUudised,Maailm,['rass'],['mustanahali'],NaN
6,"Jääb vaid küsida — millal ka liibüalased käega löövad ja toimuval vabavoolus minna lasevad, kui Euroopa vaid räägib rändekriisi ohjeldamisest, ise aga valab õli tulle?Hundid inimõiguslaste nahas on juba asunud haukuma ka väljaspoole EL-i rajatavate põgenikekeskuste peale, väites, et nagunii saavat neist “teised Guantanamod”.",MH,2,A,J,22.06.2018 08:00,"Rändekriis Vahemerel — kas vaid Liibüa rannavalve töötab selle nimel, et migrandid mere lõunakaldale jääks?",UuedUudised,UuedUudised,Maailm,"['ränne', 'pagulased']","['rände', 'põgenik']",NaN
7,"Samuti nimetas ta nn pehmeid sihtmärke, mida moslemitest tsiviilvastupanuvõitlejad saavad rünnata, täites sellega d˛ihaadi ja vastupanu kohust ning toetades sellega kaudselt relvastatud pühasõdalasi, kas või näiteks süüdata metsi kuumadel suvekuudel.35 Kaost peab külvama igal pool, nii otsene kui ka kaudne terror, sabotaa˛ ja õõnestustöö on lubatud ja lausa nõutav.",MH,MH,A,J,13.12.2015 10:25,Globaalse d˛ihaadi 

In [ ]:
# dropping unneccesary columns
data = data.drop(columns=['stanceNotConsolidated', 'coderConsolidated', 'coderNotConsolidated'])

# drop the ambiguous sentences
data = data[data["stanceConsolidated"] != "MH"]

# reset index
data = data.reset_index(drop=True)

In [ ]:
# mapping stance
stance_col = data['stanceConsolidated'].map({
    "1": 'against',
    "2": 'against',
    "3": 'neutral',
    "4": 'supportive',
    "5": 'supportive',
})

# insert the stances (as 3 classes) after the 'stanceConsolidated' column
data.insert(data.columns.get_loc('stanceConsolidated') + 1, 'stance', stance_col)

In [ ]:
def fix_specials(sentence, symbol):
  # fixes encoding issues with š and ž
  if symbol in sentence:
    words = sentence.split(" ")
    for word in words:
      if symbol in word:
        s_word = word.replace(symbol, "š")
        z_word = word.replace(symbol, "ž")

        s_text = Text(s_word).tag_layer()
        s_spellcheck = SpellCheckRetagger(add_spellcheck=True)
        s_spellcheck.retag(s_text)

        z_text = Text(z_word).tag_layer()
        z_spellcheck = SpellCheckRetagger(add_spellcheck=True)
        z_spellcheck.retag(z_text)

        if z_text.spelling[0][0]:
          sentence = sentence.replace(word, z_word)
        else:
          sentence = sentence.replace(word, s_word)

  return sentence


def fix_punctuation(sentence, punct):
  # adds a space after the sentence ends
  # e.g: "... words!Next sentence ..." -> ".. words! Next sentence ..."
  if punct not in sentence:
    return sentence

  pi = 0
  p_count = sentence.count(punct)

  for _ in range(p_count):
    pi = sentence.index(punct, pi)
    letter_before = sentence[pi-1]
    ends_sentence = pi+1 == len(sentence)

    if letter_before.isalpha() and not ends_sentence:
      letter_after = sentence[pi+1]
      if letter_after.isalpha() and letter_after.isupper():
        sentence = sentence[:pi+1] + " " + sentence[pi+1:]

  return sentence


def fix_question_marks(sentence):
  # similar to previous function, but also fixes problems caused by encoding issues
  if "?" not in sentence:
    return sentence

  qi = 0
  q_count = sentence.count("?")

  for _ in range(q_count):
    qi = sentence.index("?", qi)
    letter_before_q = sentence[qi-1]
    q_ends_sentence = qi+1 == len(sentence)

    if letter_before_q.isalpha() and not q_ends_sentence:
      letter_after_q = sentence[qi+1]
      # ... words?Next -> ...words? Next
      if letter_after_q.isalpha() and letter_after_q.isupper():
        sentence = sentence[:qi+1] + " " + sentence[qi+1:]
      # ... words?.    -> ... words?
      elif letter_after_q == ".":
        sentence = sentence[:-1]
      # ... Kaczy?ski  -> Kaczyaski
      elif letter_after_q.isalpha() and letter_after_q.islower():
        new_sentence = sentence[:qi] + "a" + sentence[qi+1:]
        sentence = new_sentence

      qi += 1

  return sentence


def fix_numbers(sentence):
  # e.g. "200 000" -> "200000"
  # implemented because this was causing issues with finding the dependency parse tree length
  pattern = r'\b\d{1,3}(?:\s\d{3})*\b'
  matches = re.findall(pattern, sentence)

  for match in matches:
    num = int(''.join(match.split()))
    sentence = sentence.replace(match, str(num))

  return sentence


punct = ['"', '“', '”', '„', "‘", "‚", "«", "»", ".", "!", "?", ","]
def cut_ends(sentence):

  quote_count = 0
  for q in punct[:8]:
    quote_count += sentence.count(q)

  if (sentence.count(".") + sentence.count("?") < 2) and (quote_count % 2 == 0):
    return sentence

  word = sentence.split(" ")[-1]
  cut_word = word

  for i in range(len(word)-1, 0, -1):
    current = word[i]
    following = word[i-1]

    if following == "%":
      break

    if (current in punct or not current.isalpha()) and (following in punct or not following.isalpha()):
      cut_word = word[:i]

  # if the word ends with a quote and there isn't a period etc before that, then add one
  if cut_word[-1] in punct[:8] and cut_word[-2] not in punct[8:]:
    cut_word = cut_word[:-1] + "." + cut_word[-1]

  sentence = sentence.replace(word, cut_word)

  # if uneven number of quotes, then add one at the end
  count = 0
  for symbol in punct[:8]:
    count += sentence.count(symbol)

  if count % 2 == 1:
    sentence += '”'

  return sentence

In [ ]:
# apply fixes
data["sentence"] = data["sentence"].progress_apply(lambda s: fix_specials(s, "�"))
data["sentence"] = data["sentence"].progress_apply(lambda s: fix_specials(s, "˛"))
data["sentence"] = data["sentence"].progress_apply(lambda s: fix_question_marks(s))
data["sentence"] = data["sentence"].progress_apply(lambda s: fix_punctuation(s, "!"))
data["sentence"] = data["sentence"].progress_apply(lambda s: fix_punctuation(s, "."))
data["sentence"] = data["sentence"].progress_apply(lambda s: fix_numbers(s))
data["sentence"] = data["sentence"].progress_apply(lambda s: cut_ends(s))

  0%|          | 0/3261 [00:00<?, ?it/s][nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
100%|██████████| 3261/3261 [00:00<00:00, 147042.78it/s]


In [ ]:
# download file
from google.colab import drive
from google.colab import files

drive.mount("/content/drive")
data.to_csv("/content/drive/MyDrive/EstonianStanceDetection/notebooks/data.csv", index=False)
# files.download("data.csv")

Mounted at /content/drive


In [ ]:
# local
data.to_csv("data.csv", index=False)